# CS4035 - Cyber Data Analytics
## Lab 1 - Fraud data

### Submit on brightspace (zip file with the name Group_xx.zip)
(i) This jupyter file completed with code, plots, figures and report for each question. Additional plots and figures can be created for explanation before the end of each main question. Lab 1 contains 5 main questions, including the bonus. Write the code or explanation below each sub question. For the explanations, include what you would normally include in the report for this lab assignment, for example data pre-processing, hypothesis tested, approach, results, etc.
(ii) The libraries needed to run this file. Except for numpy, scikit-learn, pandas, matplotlib

Your peers should be able to use the readme section for instructions and be able to run this file. 

Make sure to keep your answers concise. Maximum number of words is 1000, which you can count with the code below. (You can add around 600 words since we start at around 400).

In [17]:
# If this cell does not work try running `pip install nbformat`

import io
from IPython import nbformat

nb = nbformat.read("Lab1.ipynb", nbformat.NO_CONVERT)
word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))

print("Word count:", word_count)

Word count: 368


## Group Number : 5

## Student 1 
### Name : Chaoyi Zhu
### ID : 5500524

## Student 2
### Name :
### ID :

## Readme - Provide instructions - libraries used, location of the data file, etc. Keep it short. Remember your peers will not debug your code and should be able to reproduce the exact output you provide.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import re
import os

## 1. Visualization task – 1 A4

### 1a. Plot visulations

In [ ]:
# Load the data
df_src = pd.read_csv(os.path.join('data', 'train_data.csv'))

# Number of Transactions
print("================= Number of Transactions =================")
print("Number of rows = " + str(len(df_src.index)))
print("Number of fraudulent transactions = " + str(len(df_src.loc[df_src['label'] == 1])))
print("Number of valid transactions = " + str(len(df_src.loc[df_src['label'] == 0])))

# Show part of the sample data
print("================= Show part of the sample data =================")
display(df_src.head(5))

# Preprocessing the data
# 1.Card ID
card_enc = LabelEncoder()
card_enc.fit(df_src['card_id'])
df_src['card_id'] = card_enc.transform(df_src.card_id)

# 2.IP ID
ip_enc = LabelEncoder()
ip_enc.fit(df_src['ip_id'])
df_src['ip_id'] = ip_enc.transform(df_src.ip_id)

# 3.Remove NA values
df_src.loc[df_src['cardverificationcodesupplied'].isna(),'cardverificationcodesupplied'] = False
df_src.loc[df_src['issuercountrycode'].isna(),'issuercountrycode'] = 'ZZ'
df_src.loc[df_src['shoppercountrycode'].isna(),'shoppercountrycode'] = 'ZZ'

# 4.Country code
unique_issuer_cc = df_src['issuercountrycode'].unique()
unique_shopper_cc = df_src['shoppercountrycode'].unique()
both = np.append(unique_issuer_cc, unique_shopper_cc)
df_countrycodes = pd.DataFrame(both)
unique_codes = df_countrycodes[0].unique()
enc = LabelEncoder()
enc.fit(unique_codes)
df_src['issuercountrycode'] = enc.transform(df_src.issuercountrycode)
df_src['shoppercountrycode'] = enc.transform(df_src.shoppercountrycode)

# 5.Change currency to EUR
def conv(row):
    currency_dict = {"BGN": 1.9558, "NZD": 1.6805, "ILS": 4.0448, "RUB": 72.2099, "CAD": 1.5075, "USD": 1.1218,
                     "PHP": 58.125, "CHF": 1.1437, "ZAR": 16.0224, "AUD": 1.5911, "JPY": 124.93, "TRY": 6.6913,
                     "HKD": 8.8007, "MYR": 4.6314, "THB": 35.802, "HRK": 7.413, "NOK": 9.6678, "IDR": 15953.68,
                     "DKK": 7.4646, "CZK": 25.659, "HUF": 322.97, "GBP": 0.86248, "MXN": 21.2829, "KRW": 1308.01,
                     "ISK": 136.2, "SGD": 1.5263, "BRL": 4.405, "PLN": 4.2868, "INR": 78.0615, "RON": 4.7596,
                     "CNY": 7.5541, "SEK": 10.635}
    return row['amount'] / (currency_dict[row['currencycode']]*100)
df_src['amount_eur'] = df_src.apply(lambda x: conv(x), axis=1)

# 6.Tx-variantcode
enc1 = LabelEncoder()
enc1.fit(df_src['txvariantcode'])
df_src['txvariantcode'] = enc1.transform(df_src.txvariantcode)

# 7.Currency code
enc2 = LabelEncoder()
enc2.fit(df_src['currencycode'])
df_src['currencycode'] = enc2.transform(df_src.currencycode)

# 8.Shopper interaction
enc3 = LabelEncoder()
enc3.fit(df_src['shopperinteraction'])
df_src['shopperinteraction'] = enc3.transform(df_src.shopperinteraction)

# 9.Cache accountcode_cc
df_src['accountcode'] = df_src['accountcode'].apply(lambda x: re.sub('Account','',x))
df_src['accountcode_cc'] = 0
df_src.loc[(df_src['accountcode'] == 'UK'),'accountcode_cc'] = 'GB'
df_src.loc[(df_src['accountcode'] == 'Mexico'),'accountcode_cc'] = 'MX'
df_src.loc[(df_src['accountcode'] == 'Sweden'),'accountcode_cc'] = 'SE'
df_src.loc[(df_src['accountcode'] == 'APAC'),'accountcode_cc'] = 'APAC'

# 10.Account code
enc4 = LabelEncoder()
enc4.fit(df_src['accountcode'])
df_src['accountcode'] = enc4.transform(df_src.accountcode)

# 11.Card verification code supplied
enc5 = LabelEncoder()
enc5.fit(df_src['cardverificationcodesupplied'])
df_src['cardverificationcodesupplied'] = enc5.transform(df_src.cardverificationcodesupplied)

# 12. Mail id
df_src.loc[df_src['mail_id'].str.contains('na',case=False),'mail_id'] = 'email99999'

enc6 = LabelEncoder()
enc6.fit(df_src['mail_id'])
df_src['mail_id'] = enc6.transform(df_src.mail_id)

# Validation result of CVC/CVV2
df_src.loc[df_src['cvcresponsecode'] > 2,'cvcresponsecode'] = 3

# Feature Engineering
df_src['countries_equal'] = (df_src['shoppercountrycode'] == df_src['issuercountrycode'])
df_src.loc[df_src['countries_equal'] == False,'countries_equal'] = 0
df_src.loc[df_src['countries_equal'] == True,'countries_equal'] = 1

### 1b. Describe the visualizations and provide relavant explanations of features and relations.

## 2. Imbalance task – 1 A4 – Individual

### 2a. Print ROC curves

### 2b. Analyse the performance of the classifiers. Explain which method performs best.

### 2c. Is using SMOTE a good idea? Why (not)?

## 3. Privacy task – 1 A4 – Individual

### 3a. Print ROC curves

### 3b. Analyse the performance of the classifiers. Explain which method performs best.

### 3c. Can you explain the performance difference for the different classifiers? Is it advisable to protect people’s privacy using rank-swapping? Why (not)?

## 4. Classification task – 2 A4

### 4a. Print relevant plots and metrics with clear headings.

### 4b. Explain the applied data pre-processing steps, learning algorithms, and post-processing steps or ensemble methods. Compare the performance of the two algorithms, focusing on performance criteria that are relevant in practice, use 10-fold cross-validation.

## 5. Bonus task – 1 A4

### 5a. Provide code and report below